In [2]:
import pandas as pd
import numpy as np
import re
import os
import datetime
import time

## 通过原始数据文件夹中的每一辆车的数据，计算每一辆车的SOH，并且把相应的结果都存放到结果文件夹中。

print('运行开始！')
print('运行中......')

Path="D:\company\SOH分析"  # 运行前，修改一下Path这个路径，就是“SOH分析”存放的文件夹路径！

path0=Path+'\SOH分析\数据\用于估计SOH的数据\\'
path10=Path+'\SOH分析\数据\SOH估计结果\\'


xlsxnames=os.listdir(path0)
for xlsxname in xlsxnames:
    if xlsxname.find('.xlsx')>=0:
        # print(xlsxname)
        path1=path0+"\\"+xlsxname
            
        abstable=pd.DataFrame(columns=['车号','Time','capinit ','capinitk','cap','capk','Soh','minSOC','maxSOC'])
        temptable=[]
        initsign=0
        capsign=0
        capinitk=0
        capinit=0
        capk=0
        cap=0
        soh1=0
        soh=0
       

        timelist=[]
        namelist=[]
        
        
        data_sheet=pd.read_excel(path1)
        data=pd.DataFrame(data_sheet)

        grouped=data.groupby('充电信号')

        for subgroup in grouped:
            if subgroup[0]>0:
                subtime=[]
                t1=[]
                t2=[]
                Taverage1=0
                Taverage2=0
                Tsum1=0
                Tsum2=0

                subcurrent=[]
                subsoc=[]
                maxsoc=max(subgroup[1]['soc'])
                #print(maxsoc)
                minsoc=min(subgroup[1]['soc'])
                socgap=maxsoc-minsoc
                
                Vin=max(subgroup[1]['车号'])
                
                for temp1 in subgroup[1]['absTime']:
                    subtime.append(temp1)
                mintime=subtime[0]
                
                if minsoc<50 and socgap>20:   


                    for temp1 in subgroup[1]['batteryMaxTemp']:
                        t1.append(temp1)
                    ModT1=t1[0]

                    for temp1 in subgroup[1]['batteryMinTemp']:
                        t2.append(temp1)
                    ModT2=t2[0]


                    for counter4 in range(0,len(subtime)):
                        Tsum1=t1[counter4]+Tsum1
                        Tsum2=t2[counter4]+Tsum2


                    Taverage1=Tsum1/len(subtime)
                    Taverage2=Tsum2/len(subtime)
                    #Taverage=pd.DataFrame([Taverage1,Taverage2])

                    Taveragezong=(Taverage1+Taverage2)/2



                    for temp1 in subgroup[1]['chargeAi']:
                        subcurrent.append(temp1)

                    for temp1 in subgroup[1]['soc']:
                        subsoc.append(temp1)
                    currentsum=0
                    current60=0
                    for counter3 in range(0,len(subtime)-1):
                        # time1=datetime.datetime.strptime(subtime[counter3],"%Y-%m-%d %H:%M:%S")
                        # time2=datetime.datetime.strptime(subtime[counter3+1],"%Y-%m-%d %H:%M:%S")
                        time1=subtime[counter3]
                        time2=subtime[counter3+1]
                        # gaps=(time2-time1).total_seconds()
                        gaps=(time2-time1)
                        currentsum=(subcurrent[counter3]+subcurrent[counter3+1])/2*gaps/3600+currentsum
                        if subsoc[counter3]<60.1 and subsoc[counter3]>59.5:
                            current60=currentsum
                    if maxsoc<50:
                        current60=currentsum
                    if initsign<1:
                        initsign=1
                        capinit=current60/(min(60,maxsoc)-minsoc)*100
    
                    else:
                        capinitk=(current60/(min(60,maxsoc)-minsoc))*100
                    if abs(capinitk-capinit)>0.1*capinit:
                        signal=1
                        capinit=capinit   
                    else:
                        signal=0
                        capinit=0.9*capinit+0.1*capinitk
                       

                    if maxsoc>=90 and minsoc<50 and capinit>0 and socgap>0 and (maxsoc-minsoc)>0:
                        if capsign<1:
                            capsign=1
                            capk=currentsum/socgap*100
                            cap=capk
                            soh1=(capk/capinit)*100
                        else:
                            capk=(currentsum/(maxsoc-minsoc))*100
                            cap=0.9*cap+0.1*capk
#                         if abs(capk-cap)>0.1*cap:
#                             cap=cap   
#                         else:
#                             cap=0.9*cap+0.1*capk 
#                         soh1=(cap/capinit)*100


                            if capk<capinit and (maxsoc-minsoc)>0 and capinit>0:
                                capk=(currentsum/(maxsoc-minsoc))*100
                                cap=0.9*cap+0.1*capk
                            else:
                                cap=cap
                            soh1=(cap/capinit)*100
                    soh=soh1*(1-0.02*(Taveragezong-25)/10)   # 温度修正,修正到25℃


                temptable=pd.DataFrame([Vin,mintime,capinit,capinitk,cap,capk,soh,minsoc,maxsoc]).T
                temptable.columns=['车号','Time','capinit ','capinitk','cap','capk','Soh','minSOC','maxSOC']
                abstable=pd.concat([abstable,temptable],axis=0,sort=False,ignore_index=True)

    
        wpath=path10+'\\'+xlsxname+'sohdata.xlsx'
        writer=pd.ExcelWriter(wpath,engine='openpyxl')
        abstable.to_excel(writer,sheet_name='data',index=False)
        writer.save()
        writer.close()

print('运行结束！')

运行开始！
运行中......
运行结束！
